In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [2]:
!kaggle datasets download -d jangedoo/utkface-new

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
 99% 329M/331M [00:16<00:00, 23.6MB/s]
100% 331M/331M [00:16<00:00, 20.8MB/s]


In [3]:
import zipfile
zip = zipfile.ZipFile("/content/utkface-new.zip",'r')
zip.extractall("/content")
zip.close()

In [4]:
import os
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

In [7]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'

In [8]:
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [9]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [10]:
df.shape

(23708, 3)

In [11]:
df.head()

,age,gender,img
0,5,1,5_1_4_20170116232419298.jpg.chip.jpg
1,28,1,28_1_0_20170116221705370.jpg.chip.jpg
2,54,0,54_0_0_20170104211828524.jpg.chip.jpg
3,34,1,34_1_0_20170117134932847.jpg.chip.jpg
4,1,0,1_0_2_20161219212516942.jpg.chip.jpg


In [12]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [13]:
train_df.shape

(20000, 3)

In [14]:
test_df.shape

(3708, 3)

In [15]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [16]:
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                    class_mode='multi_output')

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                  class_mode='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [17]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [18]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

94765736/94765736 [==============================] - 5s 0us/step


In [19]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 200, 200, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 206, 206, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 100, 100, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 100, 100, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [21]:
model.compile(optimizer='adam',loss={'age': 'mse','gender': 'binary_crossentropy'},metrics={'age': 'mse','gender': 'binary_crossentropy'})

In [ ]:
model.fit(train_generator,epochs=10,batch_size = 32,validation_data=test_generator)

Epoch 1/10
625/625 [==============================] - 278s 426ms/step - loss: 420.0648 - age_loss: 419.2434 - gender_loss: 0.8213 - age_mse: 419.2434 - gender_binary_crossentropy: 0.8213 - val_loss: 406.2170 - val_age_loss: 405.5240 - val_gender_loss: 0.6930 - val_age_mse: 405.5240 - val_gender_binary_crossentropy: 0.6930
Epoch 2/10
130/625 [=====>........................] - ETA: 2:58 - loss: 399.4821 - age_loss: 398.7859 - gender_loss: 0.6963 - age_mse: 398.7859 - gender_binary_crossentropy: 0.6963

In [ ]:
import matplotlib.pyplot as plt
plt.plot(model.history.history['age_loss'])
plt.plot(model.history.history['val_age_loss'])

In [ ]:
plt.plot(model.history.history['gender_loss'])
plt.plot(model.history.history['val_gender_loss'])